In [1]:
import wandb
import pandas as pd
from tqdm import tqdm

In [2]:
import re

import ast

def safe_parse(val):
    if isinstance(val, str):
        try:
            parsed = ast.literal_eval(val)
            if isinstance(parsed, dict):
                return parsed
        except Exception:
            pass
    return val

def flatten_config(config, parent_key="", sep="."):
    items = {}
    for k, v in config.items():
        parsed = safe_parse(v)
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(parsed, dict):
            items.update(flatten_config(parsed, new_key, sep=sep))
        else:
            items[new_key] = parsed
    return items

def parse_param_count(log_str):
    match = re.search(r"([\d\.]+)\s*([KMB])\s+Trainable params", log_str)
    if not match:
        return None  # Or raise an error / fallback value

    value, unit = match.groups()
    value = float(value)

    multiplier = {
        "K": 1e3,
        "M": 1e6,
        "B": 1e9,
    }.get(unit.upper(), 1)

    return int(value * multiplier)

In [3]:
# Set your W&B project
ENTITY = "ludekcizinsky"  # e.g., "myusername" or team name
PROJECT = "seizure-prediction"

api = wandb.Api()

In [4]:
runs = api.runs(f"{ENTITY}/{PROJECT}")

In [5]:
for run in runs:
    break

In [6]:
vals = run.history(samples=10000)

In [7]:
vals[~vals["val/f1_macro"].isna()]

,optim/grad_norm_postclip,train/loss,_runtime,optim/lr,val/loss,val/f1_class_0,trainer/global_step,val/f1_class_1,optim/grad_norm_preclip,epoch,train/f1_macro,_step,val/acc,val/f1_macro,train/acc,train/f1_class_1,train/f1_class_0,_timestamp
24,NaN,NaN,90.331943,NaN,0.423628,0.902896,574,0.095238,NaN,24,NaN,24,0.824615,0.499067,NaN,NaN,NaN,1.748095e+09
50,NaN,NaN,171.148362,NaN,0.484767,0.860174,1149,0.448669,NaN,49,NaN,50,0.776923,0.654421,NaN,NaN,NaN,1.748095e+09
76,NaN,NaN,251.848982,NaN,0.475894,0.863351,1724,0.465028,NaN,74,NaN,76,0.782308,0.664190,NaN,NaN,NaN,1.748095e+09
102,NaN,NaN,333.917840,NaN,0.408851,0.910944,2299,0.413994,NaN,99,NaN,102,0.845385,0.662469,NaN,NaN,NaN,1.748095e+09
128,NaN,NaN,413.993400,NaN,0.393506,0.915863,2874,0.396226,NaN,124,NaN,128,0.852308,0.656045,NaN,NaN,NaN,1.748095e+09
154,NaN,NaN,495.125075,NaN,0.394339,0.913526,3449,0.434783,NaN,149,NaN,154,0.850000,0.674154,NaN,NaN,NaN,1.748095e+09
180,NaN,NaN,575.901231,NaN,0.437327,0.906987,4024,0.187266,NaN,174,NaN,180,0.833077,0.547126,NaN,NaN,NaN,1.748096e+09
206,NaN,NaN,655.830817,NaN,0.388706,0.916630,4599,0.381877,NaN,199,NaN,206,0.853077,0.649254,NaN,NaN,NaN,1.748096e+09
232,NaN,NaN,736.662869,NaN,0.384982,0.916044,5174,0.412308,NaN,224,NaN,232,0.853077,0.664176,NaN,NaN,NaN,1.748096e+09
258,NaN,NaN,816.903189,NaN,0.392373,0.914031,5749,0.456338,NaN,249,NaN,258,0.851538,0.685185,NaN,NaN,NaN,1.748096e+09


In [8]:
# Get all runs in the project
runs = api.runs(f"{ENTITY}/{PROJECT}")

results = []

for run in tqdm(runs):
    try:

        if run.state in ["failed", "crashed"]:
            continue
        # Load history (all metrics per step/epoch)
        history = run.history(samples=10000)  # increase samples if needed
        
        if "val/f1_macro" not in history.columns:
            continue  # skip runs without F1
        
        # Find index of best val_f1
        best_idx = history["val/f1_macro"].idxmax()
        best_row = history.loc[best_idx]

        tags = run.tags

        log_files = [f for f in run.files() if f.name == "output.log"]
        log_str = log_files[0].download(replace=True).read()
        if log_files:
            train_params = parse_param_count(log_str)
        else:
            train_params = None

        results_dict = {
            "run_id": run.id,
            "run_name": run.name,
            "tags": ", ".join(tags),
            "epoch": best_row.get("epoch", best_idx),
            "val_f1_macro": best_row.get("val/f1_macro", None),
            "val_loss": best_row.get("val/loss", None),
            "val_accuracy": best_row.get("val/acc", None),
            "val_f1_class_1": best_row.get("val/f1_class_1",None),
            "val_f1_class_0": best_row.get("val/f1_class_0",None),
            "runtime_sec": run.summary.get("_runtime", None),
            "total_epochs": history["epoch"].max().item() + 1,
            "trainable_params": train_params,
            "timestamp": run.created_at,
        }

        raw_config = run._attrs.get("config", {})
        flat_config = flatten_config(raw_config)

        # Output sample
        for k, v in flat_config.items():
            if "logger" not in k and "output_dir" not in k and "launch_cmd" not in k and "repo_root" not in k and "username" not in k and "data.root" not in k:
                results_dict[k] = v

        results.append(results_dict)
    except Exception as e:
        print(f"Error with run {run.name}: {e}")

# Convert to DataFrame for sorting/filtering
df = pd.DataFrame(results)

100%|██████████| 282/282 [06:49<00:00,  1.45s/it]


In [9]:
df.drop(df[df.tags=="debug"].index,axis=0,inplace=True)
df.drop(df[df.total_epochs == 10].index,axis=0,inplace=True)

In [10]:
def get_part(tags):
    if "part1" in tags or "part_1" in tags:
        return "part_1"
    elif "part2a" in tags:
        return "part_2a"
    elif "part2b" in tags:
        return "part_2b"
    elif "part2c" in tags:
        return "part_2c"
    elif "part3" in tags:
        return "part_3"

In [11]:
df["part"] = df["tags"].apply(get_part)

## Overall

In [12]:
df.sort_values(by="val_f1_macro",ascending=False).iloc[:,:12].head(20)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
2,2td0ekfv,soft-bee-85,"baseline, fft, kaggle, lstm, part1, table1",674.0,0.880030,0.251406,0.930000,0.802603,0.957457,2635.967565,1000,88400
10,v5isln9b,mild-bee-93,"fft, lstm, part1, wd",949.0,0.878440,0.317623,0.926923,0.801670,0.955210,2647.010627,1000,88400
27,p99dbzsq,laced-feather-112,"dropout, fft, lstm, part1",874.0,0.872077,0.319549,0.923846,0.790698,0.953456,2759.538056,1000,88400
20,gtmvhqv0,fallen-microwave-103,"fft, hidden_dim, lstm, part1",549.0,0.871457,0.285607,0.924615,0.788793,0.954120,3902.806525,1000,340000
123,xe33zh99,northern-durian-248,"cnn1d_medium, kaggle, lstm, part2b",449.0,0.870581,0.345199,0.924615,0.786957,0.954206,3794.414399,1000,145000
13,lujwcq18,golden-universe-96,"fft, lstm, part1, wd",599.0,0.870289,0.215348,0.926923,0.784580,0.955998,2655.218461,1000,88400
57,s873xj2n,devout-microwave-143,"fft, num_heads, part1, table1, tencoder",999.0,0.863343,0.862400,0.923846,0.772414,0.954273,5889.564050,1000,563000
124,3lg8jawa,fluent-spaceship-252,"cnn1d_large, lstm, part2b",724.0,0.860526,0.482502,0.916154,0.772443,0.948609,3881.166162,1000,188000
23,8na956dj,sage-oath-106,"dropout, fft, lstm, part1",649.0,0.853587,0.376574,0.913846,0.759657,0.947516,2713.346854,1000,88400
26,n316o9sh,dark-durian-111,"baseline, fft, part1, tencoder",724.0,0.850470,0.969485,0.912308,0.754310,0.946629,6326.859362,1000,563000


## Part 1

In [25]:
df[(df.part == "part_1") & ((df["tags"].str.contains("lstm")) | (df["tags"].str.contains("encoder")) | (df["tags"].str.contains("conv1d")))]["epoch"].mean()

np.float64(749.6410256410256)

In [26]:
df[(df.part == "part_1") & ((df["tags"].str.contains("lstm")) | (df["tags"].str.contains("encoder")) | (df["tags"].str.contains("conv1d")))]["trainable_params"].mean()

np.float64(259856.41025641025)

In [13]:
df[(df.part == "part_1") & (df["tags"].str.contains("lstm"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
2,2td0ekfv,soft-bee-85,"baseline, fft, kaggle, lstm, part1, table1",674.0,0.880030,0.251406,0.930000,0.802603,0.957457,2635.967565,1000,88400
10,v5isln9b,mild-bee-93,"fft, lstm, part1, wd",949.0,0.878440,0.317623,0.926923,0.801670,0.955210,2647.010627,1000,88400
27,p99dbzsq,laced-feather-112,"dropout, fft, lstm, part1",874.0,0.872077,0.319549,0.923846,0.790698,0.953456,2759.538056,1000,88400
20,gtmvhqv0,fallen-microwave-103,"fft, hidden_dim, lstm, part1",549.0,0.871457,0.285607,0.924615,0.788793,0.954120,3902.806525,1000,340000
13,lujwcq18,golden-universe-96,"fft, lstm, part1, wd",599.0,0.870289,0.215348,0.926923,0.784580,0.955998,2655.218461,1000,88400
23,8na956dj,sage-oath-106,"dropout, fft, lstm, part1",649.0,0.853587,0.376574,0.913846,0.759657,0.947516,2713.346854,1000,88400
160,g41hf69e,dark-dew-328,"baseline, fft, fold_0, lstm, part1",749.0,0.843601,0.369080,0.905334,0.745342,0.941860,2753.303172,1000,88400
163,z6n9l2hy,worthy-brook-331,"baseline, fft, fold_1, lstm, part1",999.0,0.840544,0.515463,0.902332,0.741284,0.939804,2751.515632,1000,88400
165,h7pnzxs6,rich-planet-333,"baseline, fft, fold_2, lstm, part1",774.0,0.832774,0.381676,0.896329,0.729681,0.935866,2745.642053,1000,88400
30,kazlvbcw,vital-dust-116,"fft, lstm, part1, weighted_sampler",999.0,0.829907,0.548671,0.896923,0.723140,0.936673,2778.597105,1000,88400


In [14]:
df[(df.part == "part_1") & (df["tags"].str.contains("encoder"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
57,s873xj2n,devout-microwave-143,"fft, num_heads, part1, table1, tencoder",999.0,0.863343,0.862400,0.923846,0.772414,0.954273,5889.564050,1000,563000
26,n316o9sh,dark-durian-111,"baseline, fft, part1, tencoder",724.0,0.850470,0.969485,0.912308,0.754310,0.946629,6326.859362,1000,563000
46,84zm3uwa,comfy-snowball-132,"fft, part1, tencoder, wd",974.0,0.846779,0.955033,0.914615,0.744828,0.948730,6370.567888,1000,563000
35,sg1yy2h1,devout-forest-121,"fft, lr, part1, tencoder",724.0,0.846737,0.403651,0.908462,0.749474,0.944000,6349.007331,1000,563000
169,aebxdjrt,dashing-cosmos-337,"baseline, fft, fold_2, part1, tencoder",824.0,0.845200,0.879339,0.910875,0.744371,0.946029,4840.027064,1000,563000
55,flnd4l04,fragrant-energy-141,"fft, normalize, part1, tencoder",474.0,0.839254,0.941781,0.908462,0.733781,0.944728,6414.621313,1000,563000
164,fq7no0nx,hardy-sun-332,"baseline, fft, fold_1, part1, tencoder",324.0,0.835146,0.700709,0.904641,0.728111,0.942181,4787.855554,1000,563000
56,4ogfq6bc,driven-water-142,"fft, hidden_dim, part1, tencoder",849.0,0.834545,1.058803,0.904615,0.726872,0.942218,5570.094973,1000,275000
161,o4gzbh3g,denim-sun-329,"baseline, fft, fold_0, part1, tencoder",874.0,0.834013,1.206233,0.901178,0.728426,0.939599,4756.428375,1000,563000
52,4yrpsv4a,worldly-sun-138,"fft, part1, tencoder, wd",999.0,0.827652,1.173667,0.903077,0.713636,0.941667,6394.442290,1000,563000


In [15]:
df[(df.part == "part_1") & (df["tags"].str.contains("conv1d"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
170,ueba181e,vibrant-firebrand-338,"baseline, conv1d, fft, fold_1, part1",474.0,0.847948,0.656264,0.909028,0.751576,0.944319,2682.802614,1000,149000
167,18oedcex,tough-cherry-335,"baseline, conv1d, fft, fold_0, part1",374.0,0.845524,0.708809,0.910644,0.745227,0.945821,2676.453955,1000,149000
51,3ngqywle,vivid-breeze-137,"conv1d, fft, kernels, part1, table1",674.0,0.845220,0.759532,0.910769,0.744493,0.945946,2740.182147,1000,254000
45,wx5ra7zr,proud-lake-131,"conv1d, fft, hidden_dim, part1",199.0,0.843133,0.513450,0.906923,0.743100,0.943166,2954.490435,1000,577000
34,mn4tf2le,bumbling-plasma-120,"conv1d, fft, lr, part1",999.0,0.841078,0.266486,0.903846,0.741201,0.940954,2619.920066,1000,149000
49,9zz06h10,robust-grass-135,"conv1d, fft, kernels, part1",599.0,0.838104,0.768682,0.908462,0.731377,0.944831,2755.984013,1000,465000
172,h5n8xkci,smart-gorge-340,"baseline, conv1d, fft, fold_2, part1",449.0,0.835815,0.712317,0.903025,0.730769,0.940862,2697.722533,1000,149000
48,9ydhm336,stellar-dream-134,"conv1d, fft, kernels, part1",574.0,0.834807,0.944007,0.902308,0.729211,0.940404,2614.189849,1000,128000
37,oh6ya7xa,fresh-snow-123,"conv1d, fft, part1, wd",574.0,0.832336,0.622202,0.903846,0.722838,0.941833,2616.686718,1000,149000
31,kabnumgc,floral-shape-117,"conv1d, fft, lr, part1",899.0,0.831512,0.743604,0.896923,0.726531,0.936493,2609.875924,1000,149000


In [16]:
df[(df.part == "part_1") & (df["tags"].str.contains("gat"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
25,vr0u2u3q,worldly-violet-110,"fft, gat, hidden_dim, part1, table1",949.0,0.754566,0.370728,0.876154,0.581818,0.927314,4408.092679,1000,430000
11,h20bdt1r,rich-wood-94,"fft, gat, part1, wd",899.0,0.750798,0.388456,0.870769,0.577889,0.923706,3794.153726,1000,198000
4,xh9fhf70,comfy-dawn-87,"fft, gat, lr, part1",774.0,0.738268,0.377942,0.853077,0.564920,0.911615,3772.087685,1000,198000
14,gara6unb,dazzling-hill-97,"fft, gat, part1, wd",949.0,0.734562,0.369012,0.851538,0.558352,0.910772,3837.919780,1000,198000
187,vfrixc4j,devoted-capybara-357,"fft, fold_0, gat, learned_pool, part1",649.0,0.734520,0.432012,0.859617,0.552283,0.916758,3492.618042,1000,218000
190,xmdu9ya6,sandy-sea-360,"fft, fold_1, gat, learned_pool, part1",774.0,0.733785,0.676403,0.863311,0.548092,0.919478,3447.972215,1000,218000
149,fbv6jyow,visionary-planet-315,"fft, gat, learned_pool, part1",699.0,0.733355,0.518170,0.869231,0.543011,0.923698,3576.004542,1000,218000
53,ysr1v1jo,expert-darkness-139,"fft, gat, part1, weighted_sampler",574.0,0.732460,0.413294,0.842308,0.561028,0.903891,3918.134046,1000,198000
171,bqjk0nmq,volcanic-firebrand-339,"baseline, fft, fold_1, gat, part1",574.0,0.727485,0.375386,0.848072,0.546207,0.908763,3138.975731,1000,198000
47,gvekxh9m,fearless-sea-133,"distance_threshold, fft, gat, part1",424.0,0.725436,0.493470,0.846154,0.543379,0.907493,4423.609002,1000,198000


In [17]:
df[(df.part == "part_1") & (df["tags"].str.contains("gcn"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
148,pf6cdait,fast-river-314,"fft, gcn, learned_pool, part1",699.0,0.800123,1.041971,0.889231,0.666667,0.933579,3295.364610,1000,373000
154,nks5mmpf,gallant-sun-322,"fft, gcn, kaggle, learned_pool, part1, table1",699.0,0.800123,1.041971,0.889231,0.666667,0.933579,3377.416183,1000,373000
181,dp83oglc,sandy-smoke-349,"fft, fold_2, gcn, learned_pool, part1",899.0,0.787842,1.074899,0.875317,0.651613,0.924072,2682.039361,1000,373000
175,s495lcgm,desert-shape-343,"fft, fold_0, gcn, learned_pool, part1",749.0,0.785672,1.016277,0.875317,0.647059,0.924285,3319.402295,1000,373000
147,wdixxj66,lunar-bee-313,"fft, gcn, gcn_0_5_dist, learned_pool, part1",174.0,0.783209,0.791914,0.873077,0.643629,0.922789,2936.468164,1000,373000
179,wvey8vf0,treasured-bee-347,"fft, fold_1, gcn, learned_pool, part1",324.0,0.777667,0.385703,0.873470,0.631720,0.923613,3090.034522,1000,373000
68,anjw34ub,scarlet-resonance-159,"fft, gcn_0_5_dist, hidden_dim, part1",949.0,0.774556,0.376829,0.876923,0.622642,0.926471,2683.286364,1000,62100
73,33lcxzg1,lilac-haze-164,"fft, gcn_0_5_dist, part1, pooling",774.0,0.759586,0.371426,0.870000,0.596659,0.922513,2667.857341,1000,26900
177,j1m6bnz5,iconic-paper-345,"fft, fold_1, gcn, max_pool, part1",799.0,0.759229,0.350843,0.860079,0.603403,0.915055,3012.771081,1000,62100
41,0cubreec,still-wildflower-127,"fft, gcn, part1, weighted_sampler",874.0,0.756934,0.378826,0.846923,0.609037,0.904830,3314.047066,1000,26900


## Part 2A

In [18]:
df[(df.part == "part_2a") & (df["tags"].str.contains("lstm"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
83,3pz4rcp5,frosty-oath-175,"dct_downsample, kaggle, lstm, part2a",849.0,0.809426,0.937859,0.886923,0.687898,0.930954,2440.230735,1000,88400
95,ai7jideq,azure-valley-187,"lstm, part2a, wavelet",974.0,0.802133,0.663091,0.886923,0.672606,0.931660,1252.736297,1000,88400
77,515jflw9,olive-elevator-168,"lstm, part2a, window_downsample",924.0,0.795562,1.120305,0.884615,0.660634,0.930491,1323.405299,1000,88400
81,27s17eqd,robust-wave-173,"dct_downsample, lstm, part2a",99.0,0.790036,0.369083,0.890000,0.645161,0.934911,3062.517461,1000,88400
79,s1j72ggx,deep-resonance-170,"lstm, part2a, window_downsample",949.0,0.782128,1.151947,0.874615,0.640177,0.924080,866.376918,1000,88400
78,u4isd9og,prime-lion-169,"lstm, part2a, window_downsample",74.0,0.781933,0.425083,0.876923,0.638009,0.925857,1011.316697,1000,88400
76,35cumlml,rosy-energy-167,"lstm, part2a, window_downsample",824.0,0.780858,0.836489,0.879231,0.634033,0.927683,1437.813789,1000,88400
96,ejgxtio2,morning-wind-188,"lstm, part2a, wavelet",499.0,0.780409,0.898344,0.873846,0.637168,0.923650,1033.490723,1000,88400
70,hxwwgwia,lucky-wave-161,"decimate, lstm, part2a",99.0,0.775522,0.501352,0.873077,0.627540,0.923505,942.991109,1000,88400
93,mdhyymey,balmy-grass-185,"lstm, part2a, wavelet",999.0,0.774264,0.764460,0.870769,0.626667,0.921860,1706.394955,1000,88400


In [19]:
df[(df.part == "part_2a") & (df["tags"].str.contains("gcn"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
88,o96gxicw,sleek-planet-180,"decimate, gcn, part2a",149.0,0.673085,1.962007,0.805385,0.465116,0.881053,2302.479808,1000,93600
82,yzrzpebq,upbeat-tree-174,"decimate, gcn, part2a",324.0,0.668214,1.741350,0.813077,0.448980,0.887448,5214.090533,1000,400000
99,18q4ju3d,breezy-eon-191,"gcn, part2a, window_downsample",74.0,0.662317,0.650667,0.818462,0.432692,0.891941,1899.028225,1000,48800
110,49yw8t66,copper-lake-202,"gcn, part2a, wavelet",624.0,0.660021,4.053404,0.808462,0.435374,0.884669,1914.018708,1000,41600
91,hyebwt66,soft-deluge-183,"decimate, gcn, part2a",74.0,0.659300,0.648393,0.811538,0.431555,0.887045,1823.730414,1000,42400
98,qfq2v8vd,different-yogurt-190,"gcn, part2a, window_downsample",249.0,0.656485,2.490861,0.802308,0.432671,0.880298,1950.781189,1000,55200
85,9l118qut,jumping-haze-177,"decimate, gcn, part2a",699.0,0.655642,5.694632,0.813077,0.422803,0.888481,3462.054785,1000,208000
94,en1nbwtu,likely-pond-186,"gcn, part2a, window_downsample",399.0,0.655428,3.677298,0.784615,0.444444,0.866412,3454.897880,1000,208000
107,ehfsng4j,magic-sky-199,"gcn, part2a, wavelet",99.0,0.652516,0.884255,0.790769,0.433333,0.871698,3578.393610,1000,209000
97,py4olxej,robust-wind-189,"gcn, part2a, window_downsample",99.0,0.651481,1.195303,0.771538,0.446927,0.856035,2324.710297,1000,93600


## Part 2B

In [20]:
df[(df.part == "part_2b") & (df["tags"].str.contains("lstm"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
123,xe33zh99,northern-durian-248,"cnn1d_medium, kaggle, lstm, part2b",449.0,0.870581,0.345199,0.924615,0.786957,0.954206,3794.414399,1000,145000
124,3lg8jawa,fluent-spaceship-252,"cnn1d_large, lstm, part2b",724.0,0.860526,0.482502,0.916154,0.772443,0.948609,3881.166162,1000,188000
121,2yg9l5qs,winter-aardvark-238,"cnn1d_small, lstm, part2b",674.0,0.841447,0.478782,0.909231,0.737778,0.945116,3755.898387,1000,124000
119,hkaxkasi,misty-microwave-231,"lstm, non_over_lap, part2b, patch_tst",124.0,0.801727,0.422459,0.887692,0.671171,0.932282,15595.191129,250,239000
126,9ctlopl1,electric-silence-272,"lstm, over_lap, part2b, patch_tst",49.0,0.795830,0.324695,0.887692,0.658879,0.932781,15475.687148,250,239000


In [21]:
df[(df.part == "part_2b") & (df["tags"].str.contains("gcn"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
130,4jvl7gra,twilight-leaf-281,"cnn1d_large, gcn, part2b",74.0,0.810259,0.405038,0.903846,0.677003,0.943516,4250.769686,1000,155000
157,n3o1qled,zesty-shape-325,"cnn1d_large, gcn, kaggle, part2b",74.0,0.805992,0.347378,0.896923,0.673171,0.938813,4212.047623,1000,155000
129,5uv5z81u,eager-hill-280,"cnn1d_medium, gcn, part2b",49.0,0.800159,0.296992,0.896154,0.661654,0.938664,4202.100068,1000,112000
127,az516un9,fine-glade-278,"cnn1d_small, gcn, part2b",49.0,0.797713,0.299791,0.880769,0.668094,0.927332,4179.266001,1000,91300
122,obexylo6,clean-jazz-244,"gcn, non_over_lap, part2b, patch_tst",24.0,0.660996,0.487126,0.818462,0.429952,0.892040,15600.503123,250,206000
128,hn0eexuq,zany-water-279,"gcn, over_lap, part2b, patch_tst",74.0,0.660021,3.931639,0.808462,0.435374,0.884669,15445.699140,250,206000


## Part 2C

In [22]:
df[(df.part == "part_2c") & (df["tags"].str.contains("gcn"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
140,jq32zp6r,dulcet-lion-299,"lstm, part2c, small_overlap, table_2c, window_gcn",147.0,0.808523,0.402236,0.889231,0.684211,0.932836,10110.163708,250,148000
137,lgiwxp18,solar-firefly-296,"kaggle, lstm, part2c, part2d, table_2c, weight...",49.0,0.805219,0.304460,0.896923,0.671569,0.938869,3358.505183,50,148000
141,6qfnvqyf,super-sun-303,"large_window, lstm, part2c, table_2c, window_gcn",197.0,0.800725,0.598685,0.886923,0.669663,0.931787,8229.069116,250,164000
139,6eczkgp7,misunderstood-donkey-298,"lstm, no_overlap, part2c, table_2c, window_gcn",235.0,0.800020,0.533650,0.881538,0.672340,0.927700,8520.879743,250,148000
138,pxbwzcz3,pleasant-cloud-297,"kaggle, lstm, part2c, weighted_sampler, window...",39.0,0.791536,0.315073,0.876923,0.658120,0.924953,3375.339607,50,148000
142,0r3q73zj,kind-spaceship-304,"large_window, lstm, part2c, small_overlap, tab...",107.0,0.788672,0.460802,0.879231,0.650334,0.927011,5316.964903,250,164000
135,vzzz3np6,usual-durian-294,"baseline, lstm, part2c, window_gcn",41.0,0.785011,0.311763,0.883077,0.639810,0.930211,3353.910635,50,148000
143,xe3i8n7s,restful-star-305,"large_window, lstm, no_overlap, part2c, table_...",83.0,0.774562,0.410758,0.868462,0.629067,0.920056,4622.703426,250,164000
136,299k563a,wild-vortex-295,"lr, lstm, part2c, window_gcn",45.0,0.743688,0.354738,0.857692,0.572748,0.914629,3367.280622,50,148000


In [28]:
df[(df["total_epochs"] == 50)]["epoch"].mean()

np.float64(36.36363636363637)

In [29]:
df[(df["total_epochs"] == 50)]["trainable_params"].mean()

np.float64(289636.36363636365)

## Part 3

In [23]:
df[(df.part == "part_3") & (df["tags"].str.contains("gcn"))].iloc[:,:12].sort_values(by="val_f1_macro",ascending=False)

,run_id,run_name,tags,epoch,val_f1_macro,val_loss,val_accuracy,val_f1_class_1,val_f1_class_0,runtime_sec,total_epochs,trainable_params
158,fv9amftf,gentle-violet-326,"kaggle, learned_pool, lstm_att, part2d, part3a...",47.0,0.847316,0.414346,0.915385,0.745370,0.949262,3079.260911,50,464000
152,x5m36lad,chocolate-lake-320,"correlation_adj, learned_pool, lstm_att, part3...",39.0,0.812544,0.458881,0.892308,0.690265,0.934823,16194.809829,50,464000
213,azhji2zo,fresh-blaze-396,"dist_1_5, fold_1, lstm_att learned_pool, part3...",45.0,0.792925,0.314149,0.881090,0.657807,0.928042,3465.799217,50,153000
153,4u9d3h1e,peach-snowball-321,"learned_adj, learned_pool, lstm_att, part3a, w...",1.0,0.449153,0.478395,0.815385,0.000000,0.898305,4638.535955,50,464000
